In [1]:

# lstm model
import numpy as np
import pandas as pd 
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from keras.utils import to_categorical
from matplotlib import pyplot


In [2]:
INPUT_SIGNAL_TYPES = [
    "body_acc_x_",
    "body_acc_y_",
    "body_acc_z_",
    "body_gyro_x_",
    "body_gyro_y_",
    "body_gyro_z_",
    "total_acc_x_",
    "total_acc_y_",
    "total_acc_z_"
]

# Output classes to learn how to classify
LABELS = [
    "WALKING",
    "UPSTAIRS",
    "DOWNSTAIRS",
    "SITTING",
    "STANDING",
    "LAYING"
]

In [3]:
DATASET_PATH = "/Stage 4A/UCI HAR Dataset/"

In [4]:
TRAIN = "train/"
TEST = "test/"


# Load "X" (the neural network's training and testing inputs)

def load_X(X_signals_paths):
    X_signals = []

    for signal_type_path in X_signals_paths:
        file = open(signal_type_path, 'r')
        # Read dataset from disk, dealing with text files' syntax
        X_signals.append(
            [np.array(serie, dtype=np.float32) for serie in [
                row.replace('  ', ' ').strip().split(' ') for row in file
            ]]
        )
        file.close()

    return np.transpose(np.array(X_signals), (1, 2, 0))

X_train_signals_paths =  [
    DATASET_PATH + TRAIN + "Inertial Signals/" + signal + "train.txt" for signal in INPUT_SIGNAL_TYPES
]
X_test_signals_paths = [
    DATASET_PATH + TEST + "Inertial Signals/" + signal + "test.txt" for signal in INPUT_SIGNAL_TYPES
]

X_train = load_X(X_train_signals_paths)
X_test = load_X(X_test_signals_paths)


# Load "y" (the neural network's training and testing outputs)

def load_y(y_path):
    file = open(y_path, 'r')
    # Read dataset from disk, dealing with text file's syntax
    y_ = np.array(
        [elem for elem in [
            row.replace('  ', ' ').strip().split(' ') for row in file
        ]],
        dtype=np.int32
    )
    file.close()

    # Substract 1 to each output class for friendly 0-based indexing
    return y_ - 1

y_train_path = DATASET_PATH + TRAIN + "y_train.txt"
y_test_path = DATASET_PATH + TEST + "y_test.txt"

y_train = load_y(y_train_path)
y_test = load_y(y_test_path)

In [5]:
X_train.shape


(7352, 128, 9)

In [6]:
print(y_test)

[[4]
 [4]
 [4]
 ...
 [1]
 [1]
 [1]]


In [7]:
def flatten(l):
    return [item for sublist in l for item in sublist]
y_train = flatten(y_train)
y_test = flatten(y_test)

In [11]:
y_train = pd.get_dummies(y_train)
y_test = pd.get_dummies(y_test)

In [12]:
print(X_train.shape)
print(y_train)


(7352, 128, 9)
      0  1  2  3  4  5
0     0  0  0  0  1  0
1     0  0  0  0  1  0
2     0  0  0  0  1  0
3     0  0  0  0  1  0
4     0  0  0  0  1  0
...  .. .. .. .. .. ..
7347  0  1  0  0  0  0
7348  0  1  0  0  0  0
7349  0  1  0  0  0  0
7350  0  1  0  0  0  0
7351  0  1  0  0  0  0

[7352 rows x 6 columns]


In [13]:
verbose, epochs, batch_size = 1, 25, 64
n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], y_train.shape[1]

In [14]:


	# reshape data into time steps of sub-sequences
n_steps, n_length = 4,32
trainX = X_train.reshape((X_train.shape[0], n_steps, n_length, n_features))
testX = X_test.reshape((X_test.shape[0], n_steps, n_length, n_features))
	# define model

In [2]:
from keras.layers import ConvLSTM2D
	# define model
from keras.layers import TimeDistributed
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D


model = Sequential()
model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3, activation='relu'), input_shape=(None,n_length,n_features)))
model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3, activation='relu')))
model.add(TimeDistributed(Dropout(0.5)))
model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
model.add(TimeDistributed(Flatten()))
model.add(LSTM(100))
model.add(Dropout(0.5))
model.add(Dense(100, activation='relu'))
model.add(Dense(n_outputs, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	# fit network
model.fit(trainX, y_train , epochs=epochs, batch_size=batch_size, verbose=verbose)
	# evaluate model
_, accuracy = model.evaluate(testX, y_test, batch_size=batch_size, verbose=0)

NameError: name 'Sequential' is not defined